In [1]:
import os

IMAGE_PATH = []

#get all the image paths
for root,dirs,files in os.walk("C:\\Projects\\jopan project\\Tensorflow\\workspace\\images\\test"):

    for file in files:
        if(file.endswith(".jpg")):
            IMAGE_PATH.append(os.path.join(root,file))


In [2]:
len(IMAGE_PATH)

12

In [6]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf

PATH_TO_CFG = "C:\\Projects\\jopan project\\Tensorflow\\workspace\\models\\Mobilenet640\\pipeline.config"
PATH_TO_CKPT = "C:\\Projects\\jopan project\\Tensorflow\\workspace\\models\\Mobilenet640"

print('Loading model... ', end='')
start_time = time.time()

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-12')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 1.0406033992767334 seconds


In [7]:
category_index = label_map_util.create_category_index_from_labelmap("C:\\Projects\\jopan project\\Tensorflow\\workspace\\annotations\\label_map.pbtxt",
                                                                    use_display_name=True)

In [8]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
import warnings
import time
warnings.filterwarnings('ignore')  

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))


OUTPUT_DIR = "C:\\Projects\\jopan project\\op"
os.makedirs(OUTPUT_DIR, exist_ok=True)  

latency = []

for image_path in IMAGE_PATH:
    print('Running inference for {}... '.format(image_path), end='')
    begin_time = float(time.time())

    # Load image
    image_np = load_image_into_numpy_array(image_path)

    # Convert to tensor
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

    # Perform detection
    detections = detect_fn(input_tensor)

    
    final_time = time.time()
    

    # Extract detections
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Cast detection_classes to int64
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Add label offset
    label_id_offset = 1
    image_np_with_detections = image_np.copy()



    # Visualize detections on image
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] ,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.5,
        agnostic_mode=False)
    
    
    # Save the image with detections
    output_path = os.path.join(OUTPUT_DIR, os.path.basename(image_path))
    Image.fromarray(image_np_with_detections).save(output_path)
    latency.append(final_time - begin_time)
    print('Saved to {}'.format(output_path))

print("Inference completed and images saved.")


Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy25.jpg... Saved to C:\Projects\jopan project\op\healthy25.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy26.jpg... Saved to C:\Projects\jopan project\op\healthy26.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy27.jpg... Saved to C:\Projects\jopan project\op\healthy27.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy28.jpg... Saved to C:\Projects\jopan project\op\healthy28.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy29.jpg... Saved to C:\Projects\jopan project\op\healthy29.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\healthy30.jpg... Saved to C:\Projects\jopan project\op\healthy30.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\unhealthy25.jpg... 

Saved to C:\Projects\jopan project\op\unhealthy28.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\unhealthy29.jpg... Saved to C:\Projects\jopan project\op\unhealthy29.jpg
Running inference for C:\Projects\jopan project\Tensorflow\workspace\images\test\unhealthy30.jpg... Saved to C:\Projects\jopan project\op\unhealthy30.jpg
Inference completed and images saved.


In [9]:
def calculate_fps(latency_arr):

    fps = []
    for a in latency_arr[2:]:
        f = 1/a
        fps.append(f)

    return fps

In [10]:
mean_latency = np.mean(latency)
mean_fps = np.mean(calculate_fps(latency))

In [11]:
mean_latency

2.5053529739379883

In [12]:
mean_fps

2.2634815512160253

In [12]:
lar = np.array(latency)

In [15]:
arr = np.array(latency)